In [19]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import pickle
import os
import torchvision.transforms as transforms
from pipnet.pipnet import PIPNet, get_network
from util.data import get_dataloaders
from util.vis_pipnet import get_img_coordinates
from util.func import get_patch_size
from util.eval_cub_csv import get_topk_cub
from PIL import ImageFont, Image, ImageDraw as D
from pipnet.train import test_pipnet, train_pipnet
from omegaconf import OmegaConf
from util.phylo_utils import construct_phylo_tree, construct_discretized_phylo_tree
from util.args import get_args, save_args, get_optimizer_nn
import wandb

print(torch.cuda.is_available())

!which python

True
/home/harishbabu/.conda/envs/hpnet1/bin/python


In [20]:
# run_path = '/home/harishbabu/projects/PIPNet/runs/004-CUB-27-imgnet_cnext26_img=224_nprotos=200'
run_path = '/home/harishbabu/projects/PIPNet/runs/005-CUB-27-imgnet_cnext26_img=224_nprotos=50'

device = torch.device('cuda')
device_ids = [torch.cuda.current_device()]

# device = torch.device('cpu')
# device_ids = []

args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

ckpt_path = os.path.join(run_path, 'checkpoints', 'net_trained_last')
checkpoint = torch.load(ckpt_path, map_location=device)

In [21]:
if args.phylo_config:
    phylo_config = OmegaConf.load(args.phylo_config)

if args.phylo_config:
    # construct the phylo tree
    if phylo_config.phyloDistances_string == 'None':
        root = construct_phylo_tree(phylo_config.phylogeny_path)
        print('-'*25 + ' No discretization ' + '-'*25)
    else:
        root = construct_discretized_phylo_tree(phylo_config.phylogeny_path, phylo_config.phyloDistances_string)
        print('-'*25 + ' Discretized ' + '-'*25)
else:
    # construct the tree (original hierarchy as described in the paper)
    root = Node("root")
    root.add_children(['animal','vehicle','everyday_object','weapon','scuba_diver'])
    root.add_children_to('animal',['non_primate','primate'])
    root.add_children_to('non_primate',['African_elephant','giant_panda','lion'])
    root.add_children_to('primate',['capuchin','gibbon','orangutan'])
    root.add_children_to('vehicle',['ambulance','pickup','sports_car'])
    root.add_children_to('everyday_object',['laptop','sandal','wine_bottle'])
    root.add_children_to('weapon',['assault_rifle','rifle'])
    # flat root
    # root.add_children(['scuba_diver','African_elephant','giant_panda','lion','capuchin','gibbon','orangutan','ambulance','pickup','sports_car','laptop','sandal','wine_bottle','assault_rifle','rifle'])
root.assign_all_descendents()

------------------------- No discretization -------------------------


In [22]:
# Obtain the dataset and dataloaders
trainloader, trainloader_pretraining, trainloader_normal, trainloader_normal_augment, projectloader, testloader, test_projectloader, classes = get_dataloaders(args, device)
if len(classes)<=20:
    if args.validation_size == 0.:
        print("Classes: ", testloader.dataset.class_to_idx, flush=True)
    else:
        print("Classes: ", str(classes), flush=True)

# Create a convolutional network based on arguments and add 1x1 conv layer
feature_net, add_on_layers, pool_layer, classification_layers, num_prototypes = get_network(len(classes), args, root=root)
   
# Create a PIP-Net
net = PIPNet(num_classes=len(classes),
                    num_prototypes=num_prototypes,
                    feature_net = feature_net,
                    args = args,
                    add_on_layers = add_on_layers,
                    pool_layer = pool_layer,
                    classification_layers = classification_layers,
                    num_parent_nodes = len(root.nodes_with_children()),
                    root = root
                    )
net = net.to(device=device)
net = nn.DataParallel(net, device_ids = device_ids)    
net.load_state_dict(checkpoint['model_state_dict'],strict=True)
criterion = nn.NLLLoss(reduction='mean').to(device)

Num classes (k) =  27 ['cub_001_Black_footed_Albatross', 'cub_011_Rusty_Blackbird', 'cub_016_Painted_Bunting', 'cub_019_Gray_Catbird', 'cub_030_Fish_Crow'] etc.
Number of prototypes:  50


In [17]:
epoch = 0
run = wandb.init(project="pipnet", name=os.path.basename(args.log_dir), config=vars(args), reinit=False)
info = test_pipnet(net, testloader, criterion, epoch, device, progress_prefix= 'Test Epoch', wandb_logging=True, wandb_log_subdir = 'test')
print('test', info['fine_accuracy'])
info = test_pipnet(net, trainloader, criterion, epoch, device, progress_prefix= 'Train Epoch', wandb_logging=False, wandb_log_subdir = 'train')
print('train', info['fine_accuracy'])


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: harishbabu. Use `wandb login --relogin` to force relogin


Test Epoch0: 100% 13/13 [00:07<00:00,  1.69it/s, L:21.442,LC:0.031, LA:0.01, LT:0.411]

	Fine accuracy: 1 2
	Node name: root, acc: 99.36, f1:99.34, samples: 1562, 113+001+068=1500/1502=1.0, cub_090_Red_breasted_Merganser=52/60=0.87
	Node name: 113+001+068, acc: 98.14, f1:98.13, samples: 1502, 113+060=1252/1264=0.99, 001+052=166/178=0.93, cub_068_Ruby_throated_Hummingbird=56/60=0.93
	Node name: 113+060, acc: 99.21, f1:99.21, samples: 1264, 113+187=1082/1086=1.0, 060+071=172/178=0.97
	Node name: 001+052, acc: 100.0, f1:100.0, samples: 178, 001+033=118/118=1.0, cub_052_Pied_billed_Grebe=60/60=1.0
	Node name: 113+187, acc: 99.26, f1:99.28, samples: 1086, 113+037=978/986=0.99, 187+079=100/100=1.0
	Node name: 060+071, acc: 92.13, f1:91.94, samples: 178, 060+143=116/118=0.98, cub_071_Long_tailed_Jaeger=48/60=0.8
	Node name: 001+033, acc: 96.61, f1:96.61, samples: 118, cub_001_Black_footed_Albatross=58/60=0.97, cub_033_Yellow_billed_Cuckoo=56/58=0.97
	Node name: 113+037, acc: 98.99, f1:98.97, samples: 986, 113+030=868/868=1.0, 037+077=108/118=0.92
	Node name: 187+079, acc: 92.0, 


Train Epoch0: 100% 13/13 [00:08<00:00,  1.56it/s, L:3.514,LC:0.005, LA:0.01, LT:0.041]

	Fine accuracy: 1 2
	Node name: root, acc: 100.0, f1:100.0, samples: 1620, 113+001+068=1560/1560=1.0, cub_090_Red_breasted_Merganser=60/60=1.0
	Node name: 113+001+068, acc: 100.0, f1:100.0, samples: 1560, 113+060=1320/1320=1.0, 001+052=180/180=1.0, cub_068_Ruby_throated_Hummingbird=60/60=1.0
	Node name: 113+060, acc: 100.0, f1:100.0, samples: 1320, 113+187=1140/1140=1.0, 060+071=180/180=1.0
	Node name: 001+052, acc: 100.0, f1:100.0, samples: 180, 001+033=120/120=1.0, cub_052_Pied_billed_Grebe=60/60=1.0
	Node name: 113+187, acc: 100.0, f1:100.0, samples: 1140, 113+037=1020/1020=1.0, 187+079=120/120=1.0
	Node name: 060+071, acc: 98.89, f1:98.88, samples: 180, 060+143=120/120=1.0, cub_071_Long_tailed_Jaeger=58/60=0.97
	Node name: 001+033, acc: 98.33, f1:98.33, samples: 120, cub_001_Black_footed_Albatross=58/60=0.97, cub_033_Yellow_billed_Cuckoo=60/60=1.0
	Node name: 113+037, acc: 100.0, f1:100.0, samples: 1020, 113+030=900/900=1.0, 037+077=120/120=1.0
	Node name: 187+079, acc: 94.17, f1:9

In [10]:
optimizer_net, optimizer_classifier, params_to_freeze, params_to_train, params_backbone = get_optimizer_nn(net, args)            
if args.epochs<=30:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=5, eta_min=0.001, T_mult=1, verbose=False)
else:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=10, eta_min=0.001, T_mult=1, verbose=False)

scheduler_net = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_net, T_max=len(trainloader)*args.epochs, eta_min=args.lr_net/100.)
train_info = train_pipnet(net, trainloader, optimizer_net, optimizer_classifier, scheduler_net, scheduler_classifier, criterion, epoch, args.epochs, device, pretrain=False, finetune=False)


chosen network is convnext


Train Epoch0:   0% 0/13 [00:00<?, ?it/s]

Number of parameters that require gradient:  206
Align weight:  5.0 , U_tanh weight:  2.0 Class weight: 2.0
Pretrain? False Finetune? False


Train Epoch0:   0% 0/13 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 154.00 MiB (GPU 0; 15.78 GiB total capacity; 14.51 GiB already allocated; 117.69 MiB free; 14.59 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [23]:
print(root)

root
	113+001+068
		113+060
			113+187
				113+037
					113+030
						113+085
							113+194
								113+118
									113+034
										113+016
											113+165
												113+011
													113+122
														cub_113_Baird_Sparrow
														cub_122_Harris_Sparrow
													011+097
														cub_011_Rusty_Blackbird
														cub_097_Orchard_Oriole
												165+181
													165+161
														cub_165_Chestnut_sided_Warbler
														cub_161_Blue_winged_Warbler
													cub_181_Worm_eating_Warbler
											cub_016_Painted_Bunting
										cub_034_Gray_crowned_Rosy_Finch
									cub_118_House_Sparrow
								194+019
									cub_194_Cactus_Wren
									cub_019_Gray_Catbird
							cub_085_Horned_Lark
						030+156
							cub_030_Fish_Crow
							cub_156_White_eyed_Vireo
					037+077
						cub_037_Acadian_Flycatcher
						cub_077_Tropical_Kingbird
				187+079
					cub_187_American_Three_toed_Woodpecker
					cub_079_Belted_Kingfisher
			060+071
	